In [1]:
from langchain_community.document_loaders import UnstructuredHTMLLoader

from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
import os
import dotenv

dotenv.load_dotenv()

base_path = os.getenv("BASE_PATH", ".")
API_KEY = os.getenv("OPENAI_API_KEY", "your-openai-api-key")
print(base_path)
embedding = OpenAIEmbeddings(
    model = "text-embedding-3-small",
    openai_api_key=API_KEY,
)

vectordb_path = r"C:\Users\Sese\AI_Study_Record\RAG_AGENT\rag_0606\chroma_db"


vectorstore = Chroma(
    collection_name="html_docs",
    persist_directory=vectordb_path,
    embedding_function=embedding,
)

vectorstore.delete_collection()

vectorstore = Chroma(
    collection_name="html_docs",
    persist_directory=vectordb_path,
    embedding_function=embedding,
)


C:\Users\Sese\autosave\알렌 이론 추출\theory_texts


In [16]:
from tqdm import tqdm
import os
for path in tqdm(os.listdir(base_path)):
   html_loader = UnstructuredHTMLLoader(
      file_path=os.path.join(base_path, path),
      mode="single",
   )
   documents = html_loader.load()
   for doc in documents:
      doc.metadata['theoryId'] = path.split('_')[1]
      doc.metadata['subChapterId'] = path.split('_')[0]
   vectorstore.add_documents(documents)

 96%|█████████▌| 980/1026 [09:15<00:26,  1.76it/s]


TypeError: Invalid input object: NoneType

In [ ]:
all_files = os.listdir(base_path)
files_to_process = all_files[980:]  # 에러 났던 파일부터 처리

for path in tqdm(files_to_process):
    try:
        html_loader = UnstructuredHTMLLoader(
            file_path=os.path.join(base_path, path),
            mode="single",
        )
        documents = html_loader.load()
        for doc in documents:
            doc.metadata['theoryId'] = path.split('_')[1]
            doc.metadata['subChapterId'] = path.split('_')[0]
        vectorstore.add_documents(documents)
    except Exception as e:
        print(f"[ERROR] 처리 실패: {path}")
        import traceback
        traceback.print_exc()


In [47]:
print(len(vectorstore.get()['documents']))
data = vectorstore.get()
for i, (doc, meta) in enumerate(zip(data["documents"], data["metadatas"])):
    print(f"[{i+1}] {doc[:100]}...")  # 문서 앞부분만 출력
    print(f"    metadata: {meta['source'].split('\\')[-1]}")


1023
[1] 감염성 질환에서 주로 동반되는 증상인 발열과 불명열에 대해 다룬다. 시험에 특별히 많이 출제되는 부분은 아니다. 불명열 환자에서 다음에 해야할 검사, 조치를 묻는 문제가 주로 출제...
    metadata: 1636_3826_발열, 불명열.html
[2] 그람양성, 그람음성 세균 분류와 항생제에 대해 다룬다. 이 파트에서 문제가 직접 출제되는 경우는 거의 없지만 감염내과를 공부하는데 가장 기본이 되는 내용이므로 잘 알아두자. 항생제...
    metadata: 1636_3827_항생제.html
[3] : Sepsis

국시에서 감염질환의 비중은 매년 15% 가까이 되며, 이 중 중증 감염도 많이 출제된다. 따라서 패혈증을 잘 이해하는 것이 국시에 있어서, 더 나아가 좋은 의사가...
    metadata: 1636_4736_패혈증.html
[4] : Osteomyelitis, 뼈속질염

지역사회 감염증(community-acquired infection) 단원에서는 여러 종류의 병원체가 공통적으로 일으킬 수 있는 질환들을 ...
    metadata: 1637_3860_골수염.html
[5] 지역사회 감염증에서 가장 흔한 설사와 식중독에 대해 다룬다. 일반적인 특징에 대해 전반적으로 살펴보고 각론으로 살모넬라, 이질, 캠필로박터, 콜레라에 대해서 자세히 다루겠다. 설사...
    metadata: 1637_3861_설사, 식중독.html
[6] : 위막성 대장염, Pseudomembranous colitis, PMC, Clostridium difficile infection, CDI

거의 매년 출제된다. 항생제 사용 과...
    metadata: 1638_3851_거짓막 결장염.html
[7] 병원감염(원내감염)과 의료관련감염에 대해 다룬다. 시험에 자주 출제되는 부분은 아니지만 임상에서 매우 중요하므로 의료관련감염의 중요성, 종류, 치료 및 예방법에 대해 잘 정리해두자...
    metadata: 1638_3852_Healt

In [48]:
import yaml
import os

# 벡터스토어와 retriever 세팅 (당신 코드와 동일)
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 10}
)

print(os.getcwd())
# YAML 파일에서 쿼리 로딩
with open("queries.yaml", "r", encoding="utf-8") as f:
    queries = yaml.safe_load(f).get("queries", [])

c:\Users\Sese\AI_Study_Record\RAG_AGENT\rag_0606


In [ ]:
query_log = []

for query in queries:
    docs = retriever.invoke(query)
    print(f"\n🔍 Query: {query}")

    result_entry = {"query": query, "results": []}
    
    for doc in docs:
        source = doc.metadata.get("source", "Unknown")
        excerpt = doc.page_content[:200].replace("\n", " ").strip()
        result_entry["results"].append({
            "source": os.path.basename(source),
            "excerpt": excerpt
        })
        print(f"  - {source}")

    query_log.append(result_entry)

# 결과 저장
with open("query_log.yaml", "w", encoding="utf-8") as f:
    yaml.dump(query_log, f, allow_unicode=True)
    
    import yaml

with open("query_log.yaml", "r", encoding="utf-8") as f:
    log_data = yaml.safe_load(f)

lines = ["# 🔍 RAG Query Log\n"]

for entry in log_data:
    query = entry.get("query", "")
    results = entry.get("results", [])

    lines.append(f"\n## 🧪 Query: **{query}**\n")
    lines.append("| Rank | Source | Excerpt |")
    lines.append("|------|--------|---------|")

    for i, result in enumerate(results, 1):
        source = result.get("source", "N/A")
        excerpt = result.get("excerpt", "").replace("|", "\\|")  # Markdown-safe
        lines.append(f"| {i} | `{source}` | {excerpt} |")

# Markdown 파일 저장
with open("query_log.md", "w", encoding="utf-8") as f:
    f.write("\n".join(lines))

print("✅ query_log.md 생성 완료!")


In [20]:
def get_target_source(target_source, max_length=None, base_path=base_path):
    target_source = os.path.join(base_path, target_source)
    # vectorstore의 전체 문서와 메타데이터 가져오기
    vectorstore_data = vectorstore.get()
    documents = vectorstore_data['documents']
    metadatas = vectorstore_data['metadatas']

    # source가 일치하는 문서 필터링
    matching_docs = [
        {"content": doc, "metadata": meta}
        for doc, meta in zip(documents, metadatas)
        if meta.get("source") == target_source
    ]

    if len(matching_docs) != 1:
        print(f"⚠️ '{target_source}'에 대해 {len(matching_docs)}개의 문서가 일치합니다. (정확히 1개여야 함)")
        return "error"

    content = matching_docs[0]['content']
    if max_length is not None:
        content = content[:max_length]

    return content


In [25]:
print(contents)

['거의 출제되지 않는 파트이다. 입원의 종류를 구분할 수 있는 것이 중요하다.\n\n1. 입원방법\n\n정신건강증진 및 정신질환자 복지서비스 지원에 관한 법률\n\n입원 종류 동의의 주체 비고 자의입원 (제41조) 환자 본인 환자 본인이 원하여 입원 동의입원 (제42조) 보호자의무자 환자는 입원의 필요성을 느낌 보호의무자에 의한 입원 (제43조) 보호자의무자 2명 이상 & 정신과 전문의 환자는 비자의적으로 입원 자치단체장에 의한 입원 (제44조) 자치단체장의 & 정신과 전문의 응급입원 (제50조) 의사 & 경찰관 정신질환자 로 자해 및 타해의 위험 이 클 때 (제41조~제44조를 적용할 시간적 여유가 없음)\n\nKaplan & Sadock 12e, Ch.30', '수술은 환자 개인의 삶에 있어서 매우 큰 사건이며, 신체적으로도 많은 변화를 동반하며, 이 중 출혈 등 몇몇은 심각한 합병증으로 환자의 예후에 큰 영향을 끼칠 수 있다. 본 단원에서는 환자의 기저질환이나 약물력에 따라 수술 전 시행해야 하는 조치들에 대해 다룬다. 특히 항혈전 약물을 제제별로 언제 끊어야 하는지, 수술 전 출혈 및 혈전 가능성에 대한 검사 해석과 조치가 많이 출제된다.\n\n1. Operability 평가 및 수술 전후 약물 관리\n\nACEi/ARB • 1일 전까지 BB, CCB, 항부정맥제 • 당일까지 기관지확장제 • 당일까지 PPI, H2 blocker • 당일까지 Insulin • 당일까지 (intermediate-acting) 경구혈당강하제 • 1일 전까지 (DPP-4i는 당일까지) 갑상샘 약물 • 당일까지 스테로이드 • 당일까지 (or 증량) 여성호르몬 관련 • 경구피임약: 4주 전까지 • HRT, raloxifene, tamoxifen: 2주 전까지 항혈소판제 ', '뱀 물림 후 처치를 고르는 문제가 출제된다. 민간에서 흔히 알고있는 처치는 대부분 오답이므로 아래 처치 항목을 주의깊게 보면 문제를 푸는 것은 어렵지 않다. 기타 동물에 의한 물림에서는 high risk에

In [28]:
import dotenv
import os
import yaml

from langchain_google_genai import ChatGoogleGenerativeAI

# 환경 변수 로드
dotenv.load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "your-gemini-api-key")

# 쿼리 로그 로드
with open("query_log.yaml", "r", encoding="utf-8") as f:
    log_data = yaml.safe_load(f)

# LLM 초기화
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    api_key=GEMINI_API_KEY,
)

# 결과 저장 리스트
augmented_log = []
markdown_lines = ["# LLM Query Results\n"]

for entry in log_data:
   query = entry.get("query", "")
   results = entry.get("results", [])
   sources = [r.get("source") for r in results if "source" in r]
   print(sources)
   contents = []
   for source in sources:
      content = get_target_source(source, max_length=2000)
      if content == "error":
         continue
      contents.append(content)
   context = "\n".join([f"{'#'*10}\n{i+1}. {content}" for i, content in enumerate(contents)])
   prompt = f"""
You are an expert in the field of computer science. Based on the following context, please answer the question:

Context:
{context}

Question: {query}

Answer:
"""
   print(prompt)

   response = llm.invoke(prompt).content.strip()

   # 로그에 응답 추가
   entry["llm_answer"] = response
   augmented_log.append(entry)

   # Markdown에 포맷 추가
   markdown_lines.append(f"## 🔍 Query\n{query}\n")
   markdown_lines.append("### 📚 Context\n<details>\n<summary>펼치기/접기</summary>\n")
   markdown_lines.append(f"\n{context}\n")
   markdown_lines.append("</details>\n")
   markdown_lines.append(f"### 💬 Answer\n{response}\n")
   markdown_lines.append("---\n")

# YAML로 저장
with open("query_log_llm.yaml", "w", encoding="utf-8") as f:
    yaml.dump(augmented_log, f, allow_unicode=True, sort_keys=False)

# Markdown으로 저장
with open("query_log_llm.md", "w", encoding="utf-8") as f:
    f.write("\n".join(markdown_lines))

print("✅ Results saved to 'query_log_llm.yaml' and 'query_log_llm.md'")


['1643_3848_여행 관련 바이러스 감염.html', '2522_5676_Leptospira interrogans.html', '2521_5659_Legionella pneumophila.html', '1637_3860_골수염.html', '2747_5639_염증의 양상.html', '2525_5685_Rubella virus.html', '1636_3826_발열, 불명열.html', '1638_3852_Healthcare-associated infections.html', '2525_5689_Norovirus.html', '1639_3828_피부연조직 감염.html']

You are an expert in the field of computer science. Based on the following context, please answer the question:

Context:
##########
1. 뎅기열, 치쿤구니야열, 지카 바이러스 감염증, 황열, 메르스에 대해서 다룬다. 여행력과 임상양상을 보고 가능성이 높은 진단명을 고르는 문제가 자주 출제된다. 비슷한 임상증상과 각 질환별 감별 포인트를 잘 기억해두자. 뎅기열, 치쿤구니야열, 지카 바이러스 감염증은 공통적으로 모기 매개 감염병이며, 뎅기열은 눈 뒤쪽 안구통, 치쿤구니야열은 관절통, 지카 바이러스 감염증은 결막염이 가장 특징적인 증상이다. 여행 관련 감염증은 이 외에도 말라리아, 황열, A형 간염, 장티푸스, 수막알균수막염, 공수병, 일본뇌염, 콜레라 등 다양한데, 유행하는 지역 여행 전 예방접종이 필요한 질환이 많고 문제에도 자주 출제되므로 성인 예방접종 단원과 연관지어 공부하자.

1. 뎅기열(Dengue fever)

1) 원인: Dengue virus

(1) Vector: 모기(Aedes aegypti)

(2) 분포: 열대지역(동남아시아, 남태평양, 아프리카, 아메리카 대륙)

(3) 해외유입 감염병 중 m/c: 국내 환자 빠르게 증가 중

2) 임상양상: HFRS처럼 바이러